## Procesamiento final de los resultados



In [1]:
import os
import json
import random
import pandas as pd
import ast 
import re
import locale
from collections import Counter
from datetime import datetime

Continua de: 005_ejemplo_uso de la carpeta colab.


In [2]:
#Establecer el locale a español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

'es_ES.UTF-8'

In [3]:
csv_path = "Data/dataset_con_resultados.csv"
df = pd.read_csv(csv_path)

# Función para convertir JSON con comillas simples a diccionario
def convertir_json_a_diccionario(json_str):
    try:
        return ast.literal_eval(json_str)
    except (ValueError, SyntaxError) as e:
        print(f"Error al convertir JSON: {e}")
        return None

# Aplicar la función a la columna 'json'
df['json'] = df['json'].apply(convertir_json_a_diccionario)
df['reglas'] = df['reglas'].apply(convertir_json_a_diccionario)


# Función para convertir la cadena de lista a una lista real
def convertir_a_lista(cadena):
    try:
        lista = ast.literal_eval(cadena)
        if isinstance(lista, list):
            return lista
    except (SyntaxError, ValueError):
        pass  # En caso de error, simplemente retorna una lista vacía
    return []


df['texto'] = df['texto'].apply(convertir_a_lista)


Esta parte del procesamiento de los resultados la tenemos que hacer en local, debido a que Google Colab da problemas con la configuración 'es_ES.UTF-8', que es necesaria en este paso.




In [4]:

# Función para extraer el valor numérico de un string y mantener coma como separador decimal
def extraer_valor_numerico(value):
    if isinstance(value, (int, float)):
        return f"{value}".replace('.', ',')
    elif isinstance(value, str):
        match = re.search(r'[-+]?\d*[.,]?\d+', value)
        if match:
            return match.group().replace('.', ',')
    return value

# Función para convertir fechas a un formato uniforme
def convertir_fecha(fecha_str):
    formatos_fecha = [
        '%d/%m/%Y', '%d.%m.%Y', '%d-%m-%Y', '%d de %B de %Y', '%d-%B-%Y',
        '%d de %b de %Y', '%d %b %Y', '%d/%m/%y', '%d-%m-%y', '%d %B, %Y', '%d %b, %Y'
    ]
    for formato in formatos_fecha:
        try:
            fecha_obj = datetime.strptime(fecha_str, formato)
            return fecha_obj.strftime('%d.%m.%Y')
        except ValueError:
            continue
    return fecha_str  # Retorna el string original si no coincide con ningún formato

# Función para transformar cada fila
def transformar_json_comprobacion(json_str):
    # Convertir el string del diccionario a un diccionario real
    diccionario = ast.literal_eval(json_str)
    
    # Procesar las claves específicas primero
    claves_especificas = ['importe_factura', 'consumo_periodo', 'potencia_contratada']
    for clave in claves_especificas:
        if clave in diccionario and diccionario[clave]:
            diccionario[clave][0] = extraer_valor_numerico(diccionario[clave][0])
    
    # Convertir fechas a formato uniforme
    claves_fechas = ['inicio_periodo', 'fin_periodo', 'fecha_cargo']
    for clave in claves_fechas:
        if clave in diccionario and diccionario[clave]:
            diccionario[clave][0] = convertir_fecha(diccionario[clave][0])
    
    # Iterar sobre los items del diccionario
    for key, value in diccionario.items():
        if key == 'consumo_periodo':
            diccionario[key] = int(float(value[0].replace(',', '.'))) if value else 0
        else:
            diccionario[key] = str(value[0]) if isinstance(value, list) and value else ""
    
    return diccionario

# Aplicar la función a cada fila del DataFrame
df['resultados'] = df['json_comprobacion'].apply(transformar_json_comprobacion)

In [5]:
# Función para modificar los valores en los diccionarios
def modificar_direcciones(diccionario):
    if 'calle_cliente' in diccionario:
        diccionario['calle_cliente'] = diccionario['calle_cliente'].replace('S/ N', 'S/N')
    if 'dirección_comercializadora' in diccionario:
        diccionario['dirección_comercializadora'] = diccionario['dirección_comercializadora'].replace('S/ N', 'S/N')
    return diccionario

# Aplicar la función a la columna 'resultados'
df['resultados'] = df['resultados'].apply(modificar_direcciones)

In [6]:
# Función para eliminar espacios innecesarios en nombre_comercializadora
def modificar_nombre_comercializadora(diccionario):
    if 'nombre_comercializadora' in diccionario:
        diccionario['nombre_comercializadora'] = re.sub(r'\s*([./-])\s*', r'\1', diccionario['nombre_comercializadora'])
    return diccionario

# Aplicar las funciones a la columna 'resultados'
df['resultados'] = df['resultados'].apply(modificar_nombre_comercializadora)

Las funciones anteriores se encargan de que los valores del diccionario tengan el formato adecuado.

In [7]:
df

,id,texto,predict,json,json_comprobacion,reglas,resultados
0,factura_0,"[DATOS, DE, LA, FACTURA, Nº, factura, :, SV504...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","{'nombre_cliente': 'Conrado Daniel Iglesias', ...",{'nombre_cliente': ['Conrado Daniel Iglesias']...,"{'nombre_cliente': [], 'dni_cliente': ['736351...","{'nombre_cliente': 'Conrado Daniel Iglesias', ..."
1,factura_2,"[DATOS, DE, LA, FACTURA, Nº, factura, :, H4623...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",{'nombre_cliente': 'BENEDICTA GALLEGOS AGUILAR...,{'nombre_cliente': ['BENEDICTA GALLEGOS AGUILA...,"{'nombre_cliente': [], 'dni_cliente': ['744343...",{'nombre_cliente': 'BENEDICTA GALLEGOS AGUILAR...
2,factura_3,"[DATOS, DE, LA, FACTURA, Nº, factura, :, SF395...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","{'nombre_cliente': 'Belinda Zetina Mijares', '...","{'nombre_cliente': ['Belinda Zetina Mijares'],...","{'nombre_cliente': [], 'dni_cliente': ['899016...","{'nombre_cliente': 'Belinda Zetina Mijares', '..."
3,factura_4,"[DATOS, DE, LA, FACTURA, IMPORTE, FACTURA, :, ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",{'nombre_cliente': 'PANTALEÓN VELASCO DE ALBA'...,{'nombre_cliente': ['PANTALEÓN VELASCO DE ALBA...,"{'nombre_cliente': [], 'dni_cliente': ['431653...",{'nombre_cliente': 'PANTALEÓN VELASCO DE ALBA'...
4,factura_5,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","{'nombre_cliente': 'SATURNINO MALTÉS NARANJO',...",{'nombre_cliente': ['SATURNINO MALTÉS NARANJO'...,"{'nombre_cliente': [], 'dni_cliente': ['137141...","{'nombre_cliente': 'SATURNINO MALTÉS NARANJO',..."
...,...,...,...,...,...,...,...
909,factura_994,"[Página, 1, /, 2, VIRGIN, ELECTRIC, SLU, CIF, ...","['O', 'O', 'O', 'O', 'O', 'B-NOMC', 'B-NOMC', ...","{'nombre_cliente': 'PETRONIO RAMÍREZ COLLADO',...",{'nombre_cliente': ['PETRONIO RAMÍREZ COLLADO'...,"{'nombre_cliente': [], 'dni_cliente': ['267273...","{'nombre_cliente': 'PETRONIO RAMÍREZ COLLADO',..."
910,factura_995,"[Página, 1, /, 2, ELECTRICA, NTRA, SRA, DE, GR...","['O', 'O', 'O', 'O', 'O', 'B-NOMC', 'B-NOMC', ...","{'nombre_cliente': 'SULPICIO ESCOVAR FONSECA',...",{'nombre_cliente': ['SULPICIO ESCOVAR FONSECA'...,"{'nombre_cliente': [], 'dni_cliente': ['209801...","{'nombre_cliente': 'SULPICIO ESCOVAR FONSECA',..."
911,factura_996,"[DATOS, DE, LA, FACTURA, Nº, factura, :, U2093...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","{'nombre_cliente': 'Petrona Uribe Naranjo', 'd...","{'nombre_cliente': ['Petrona Uribe Naranjo'], ...","{'nombre_cliente': [], 'dni_cliente': ['269342...","{'nombre_cliente': 'Petrona Uribe Naranjo', 'd..."
912,factura_997,"[DATOS, DE, LA, FACTURA, IMPORTE, FACTURA, :, ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",{'nombre_cliente': 'CELESTINA TREMINIO VALLEJO...,{'nombre_cliente': ['CELESTINA TREMINIO VALLEJ...,"{'nombre_cliente': [], 'dni_cliente': ['190202...",{'nombre_cliente': 'CELESTINA TREMINIO VALLEJO...


In [8]:
df['resultados'][900]

{'nombre_cliente': 'Ingmar Martínez Calles',
 'dni_cliente': '65080903B',
 'calle_cliente': 'Calle Yanusari Kwataba',
 'cp_cliente': '01208',
 'población_cliente': 'Barrundia',
 'provincia_cliente': 'Álava',
 'nombre_comercializadora': 'VIRGIN ELECTRIC SLU',
 'cif_comercializadora': 'B37547619',
 'dirección_comercializadora': 'C/ SEGUNDA, 4',
 'cp_comercializadora': '37188',
 'población_comercializadora': 'Arapiles',
 'provincia_comercializadora': 'Salamanca',
 'número_factura': 'Z2804917687',
 'inicio_periodo': '13.08.2001',
 'fin_periodo': '12.10.2001',
 'importe_factura': '247,91',
 'fecha_cargo': '14.10.2001',
 'consumo_periodo': 956,
 'potencia_contratada': '4,723'}

In [9]:
df['json'][900]

{'nombre_cliente': 'Ingmar Martínez Calles',
 'dni_cliente': '65080903B',
 'calle_cliente': 'Calle Yanusari Kwataba',
 'cp_cliente': '01208',
 'población_cliente': 'Barrundia',
 'provincia_cliente': 'Araba/Álava',
 'nombre_comercializadora': 'VIRGIN ELECTRIC SLU',
 'cif_comercializadora': 'B37547619',
 'dirección_comercializadora': 'C/ SEGUNDA, Nº 4',
 'cp_comercializadora': '37188',
 'población_comercializadora': 'CARBAJOSA DE LA SAGRADA',
 'provincia_comercializadora': 'Salamanca',
 'número_factura': 'Z2804917687',
 'inicio_periodo': '13.08.2001',
 'fin_periodo': '12.10.2001',
 'importe_factura': '247,91',
 'fecha_cargo': '17.10.2001',
 'consumo_periodo': 956,
 'potencia_contratada': '4,723'}

Finalmente, comparamos los resultados con el json original.

In [10]:
def calcular_porcentaje_promedio_coincidencia(df):
    coincidencias_totales = {}
    total_filas = len(df)
    
    # Iterar sobre cada fila del DataFrame
    for index, row in df.iterrows():
        json_original = row['json']
        json_transformado = row['resultados']
        
        # Comparar cada entrada de los diccionarios json y json_2
        for key, value in json_original.items():
            if key not in coincidencias_totales:
                coincidencias_totales[key] = {'total': 0, 'coincidencias': 0}
            
            # Si la clave está en ambos diccionarios y los valores coinciden, sumar 1 a las coincidencias totales
            coincidencias_totales[key]['total'] += 1
            if key in json_transformado and json_transformado[key] == value:
                coincidencias_totales[key]['coincidencias'] += 1
    
    # Calcular el porcentaje promedio de coincidencia para cada clave
    for key, values in coincidencias_totales.items():
        coincidencias_totales[key] = (values['coincidencias'] / values['total']) * 100 if values['total'] != 0 else 0
    
    return coincidencias_totales

# Uso de la función para calcular el porcentaje promedio de coincidencia entre json y json_2
porcentaje_promedio_coincidencia = calcular_porcentaje_promedio_coincidencia(df)


In [11]:
calcular_porcentaje_promedio_coincidencia(df)

{'nombre_cliente': 98.24945295404814,
 'dni_cliente': 98.57768052516411,
 'calle_cliente': 95.84245076586433,
 'cp_cliente': 99.45295404814004,
 'población_cliente': 95.40481400437638,
 'provincia_cliente': 94.20131291028446,
 'nombre_comercializadora': 75.38293216630197,
 'cif_comercializadora': 97.15536105032822,
 'dirección_comercializadora': 54.70459518599562,
 'cp_comercializadora': 99.78118161925602,
 'población_comercializadora': 62.691466083150985,
 'provincia_comercializadora': 81.50984682713349,
 'número_factura': 99.56236323851203,
 'inicio_periodo': 100.0,
 'fin_periodo': 100.0,
 'importe_factura': 94.42013129102844,
 'fecha_cargo': 75.054704595186,
 'consumo_periodo': 99.78118161925602,
 'potencia_contratada': 80.19693654266959}

In [12]:
df_resultados = df[['id', 'resultados']]

In [13]:
#Ruta para guardar el archivo CSV
csv_path = "Data/resultados_finales.csv"

# Guardar el DataFrame como un archivo CSV
df_resultados.to_csv(csv_path, index=False)#

print(f"El dataset se ha guardado en {csv_path}")

El dataset se ha guardado en Data/resultados_finales.csv


In [16]:
# Crear la carpeta 'resultados' si no existe
os.makedirs('resultados', exist_ok=True)

# Iterar sobre cada fila del DataFrame
for index, row in df_resultados.iterrows():
    # Extraer el id y resultados de la fila actual
    file_id = row['id']
    resultados = row['resultados']
    
    # Crear el nombre del archivo JSON
    filename = f'resultados/{file_id}.json'
    
    # Guardar los resultados en el archivo JSON
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(resultados, file, ensure_ascii=False, indent=4)

print("Archivos JSON creados exitosamente.")

Archivos JSON creados exitosamente.
